# Generating Random Real-Looking Words with Markov Chains

Code written by Alex Krentsel

In [1]:
import numpy as np
import string

## Read in words

In [2]:
# load_words() and words_dictionary.json taken from https://github.com/dwyl/english-words

import json
import os, sys

def load_words():
    try:
        filename = "words_dictionary.json"
        with open(filename,"r") as english_dictionary:
            valid_words = json.load(english_dictionary)
            return valid_words
    except Exception as e:
        return str(e)

In [3]:
words_dict = load_words()

## Matrix Approach

Same as a Markov chain where a state is just the current letter in the word.

In [4]:
# $ denotes start, # denotes finish
letters = 'abcdefghijklmnopqrstuvwxyz-$#'
char2num = {}
num2char = {}
for i in range(len(letters)):
    char2num[letters[i]] = i
    num2char[i] = letters[i]

In [5]:
# Entry (i, j) is number of times i goes to j
trans = np.zeros((len(letters), len(letters)))

In [6]:
START_INDEX = 27
END_INDEX = 28
def process_word(word):
    trans[START_INDEX, char2num[word[0]]] += 1
    trans[char2num[word[-1]], END_INDEX] += 1
    for i in range(len(word) - 1):
        trans[char2num[word[i]], char2num[word[i+1]]] += 1

In [8]:
for word in words_dict.keys():
    process_word(word)

In [9]:
for i in range(trans.shape[0]):
    trans[i] = trans[i] / np.sum(trans[i])

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


In [10]:
def generate_word(T):
    new_word = ""
    row_index = np.random.choice(len(letters), 1, p=T[START_INDEX])[0]
    while row_index != END_INDEX:
        new_word += num2char[row_index]
        row_index = np.random.choice(len(letters), 1, p=T[row_index])[0]
    return new_word

In [11]:
generated_words = [generate_word(trans) for _ in range(100)]

In [12]:
print(generated_words)

['rsivinolattrts', 'ckag', 'raleicthiegops', 'd', 'pancarrtinloprsmeratem', 'hride', 'flickskatdicigers', 'm', 'gonag', 'tteoutisppawsustte', 'sty', 'ckochekeylloubtoglideencumedd', 'furicoudaly', 'telelyaranakly', 'rrinthate', 'suniste', 'pmongin', 'elaearepoa', 's', 'cutenohra', 'acaldeseg', 'lblicemar', 'enesior', 'trrisexavery', 'merysk', 's', 'pen', 'shoguricti', 'ioleryon', 'phenderacsdlanisoshlimocom', 'saserwonendl', 'acothurssses', 'ckeraintirn', 'ckicantiratonctar', 'ppliarjerinc', 'terrmalonste', 'py', 'lisscknggedrube', 'mavaeunongoumel', 'rkira', 'a', 'd', 'epls', 'notedrophadrong', 'eanbeniatingysmateniveritapl', 'gly', 'tri', 'vebesolestofeseg', 'amyly', 'ex', 'epond', 'gecleomiontaumble', 'tre', 'flers', 'wc', 'dr', 'veopalasatbidinfosulensetramis', 'wscanonexdeldrmitrerumarg', 'granteroaconowlalm', 'rensstexistrt', 'modacaleracanoisc', 'm', 'asineldelericaveng', 'ti', 'churitopretichakeufaryl', 'sicencothrabod', 's', 'a', 'eppppungermmaveag', 'thlagog', 'apanghe', 'cis

## Node Approach
Markov chain with each state consisting of the current letter and the previous 2 letters (3 letters total)

In [13]:
class Node:
    def __init__(self, val):
        self.out = []
        self.out_count = []
        self.val = val
    def next_node(self):
        total = sum(self.out_count)
        probs = [val / total for val in self.out_count]
        index = np.random.choice(len(self.out), 1, p=probs)[0]
        return self.out[index]
    def add_out(self, key):
        if key in self.out:
            self.out_count[self.out.index(key)] += 1
        else:
            self.out.append(key)
            self.out_count.append(1)

In [14]:
def split_word(word):
    word = '   ' + word + '   '
    return [word[i:i+3] for i in range(len(word) - 2)]

In [15]:
def process_word_node(word, node_dict):
    keys = split_word(word)
    for i in range(len(keys) - 1):
        start_key = keys[i]
        next_key = keys[i+1]
        if start_key not in node_dict:
            node_dict[start_key] = Node(start_key)
        node_dict[start_key].add_out(next_key)

In [16]:
node_dict = {}

In [17]:
for word in words_dict.keys():
    process_word_node(word, node_dict)

In [18]:
def generate_word_node(node_dict):
    assert '   ' in node_dict, 'Must have a starting node'
    word = ''
    node = node_dict[node_dict['   '].next_node()]
    while node.val != '   ':
        word += node.val[-1]
        next_node_key = node.next_node()
        node = node_dict[next_node_key]
    return word[:-2]

In [19]:
generate_word_node(node_dict)

'bentreor'

In [20]:
node_word_list = [generate_word_node(node_dict) for _ in range(100)]

In [21]:
print(node_word_list)

['zonabilizing', 'isobed', 'metriglyptosoiding', 'moul', 'accomethnocarbongourntegocerchful', 'outh', 'cofactive', 'sorbidnexotic', 'scovid', 'unsatid', 'bolity', 'cludelio', 'missoring', 'imosemiteariencephanda', 'mycoidly', 'sperian', 'tanguisia', 'sylidae', 'unverous', 'kiness', 'bency', 'scateronar', 'hies', 'vological', 'crepenaggian', 'pretophization', 'olith', 'premultal', 'jah', 'sing', 'xenomanulicationaquamors', 'mossign', 'goosegmatic', 'globulical', 'rhondersuper', 'incupulmonographysis', 'overcipitercic', 'verm', 'marshippleliclinctuals', 'intron', 'uninizing', 'scutes', 'undes', 'reness', 'obrackles', 'apophophosaultitism', 'coat', 'capetacholourstiline', 'fibrimacross', 'mids', 'cameikh', 'loriacting', 'alaevolute', 'cuff', 'peable', 'poresack', 'interines', 'heremon', 'exsanagirr', 'emmatrix', 'chidropody', 'wood', 'improvoid', 'otomobility', 'cle', 'carancessagain', 'impolympanize', 'few', 'mastiarizing', 'overing', 'prein', 'accents', 'thold', 'curvenes', 'meazer', 'o